In [1]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/apple/Documents/Personal/Computer Vision and Deep Learning/pc-parts'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
  root_dir: Path
  updated_base_model: Path
  params_learning_rate: float

In [11]:
from pcpartsclassifier.utils.common import read_yaml, create_directories
from pcpartsclassifier.constants import *

class ConfigurationManager:
  def __init__(self) -> None:
    pass
    
  def get_base_model_config(self) -> None:
    config = self.config.base_model
    base_model_config = BaseModelConfig(
      root_dir=config.root_dir,
      params_learning_rate=self.params.LEARNING_RATE
    )
    return base_model_config


In [45]:
from torchvision import models
import torch.nn as nn


class BaseModel(nn.Module):
  def __init__(self,  output_feature: int):
    super(BaseModel, self).__init__()
    self.weights = models.EfficientNet_B0_Weights.DEFAULT
    self.transform = self.weights.transforms()
    self.model = models.efficientnet_b0(weights=self.weights)
    print(self.transform)

    for params in self.model.features.parameters():
      params.requires_grad = False

    self.model.classifier = nn.Sequential(
      nn.Dropout(),
      nn.Linear(1280, output_feature)
    )

  def forward(self, x):
    return self.model(x)


In [46]:
from torchinfo import summary
model = BaseModel(14)
model

summary(model, input_size=[1, 3, 224, 224], col_names=["input_size",  
                                                        "output_size",
                                                        "num_params",
                                                        "kernel_size",  
                                                        "trainable"])



ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Kernel Shape              Trainable
BaseModel                                                    [1, 3, 224, 224]          [1, 14]                   --                        --                        Partial
├─EfficientNet: 1-1                                          [1, 3, 224, 224]          [1, 14]                   --                        --                        Partial
│    └─Sequential: 2-1                                       [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        --                        False
│    │    └─Conv2dNormActivation: 3-1                        [1, 3, 224, 224]          [1, 32, 112, 112]         (928)                     --                        False
│    │    └─Sequential: 3-2                                  [1, 32, 112, 112]         [1, 16, 112, 112]         (1,448)                 

In [ ]:
# pipeline
try:
  model = BaseModel(14)
except Exception as e:
  raise e